In [3]:
import numpy as np
import cv2

import glob

from os import path as ospath
from utils import get_glass_height_cm

def center_crop(image, crop_width, crop_height):
    height, width, _ = image.shape

    # Calculate the center of the image
    center_x, center_y = width // 2, height // 2

    # Define the coordinates for the crop (top-left corner of the ROI)
    x1 = center_x - crop_width // 2
    y1 = center_y - crop_height // 2

    # Define the coordinates for the bottom-right corner of the ROI
    x2 = center_x + crop_width // 2
    y2 = center_y + crop_height // 2

    # Crop the image using the calculated coordinates
    return image[y1:y2, x1:x2]

CAMERA CALIBRATION FROM OPENCV SITE

In [43]:
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
 
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
 
images = glob.glob('images/chess-*.jpg')
 
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
 
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7,6), None)
 
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
 
        corners2 = cv2.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)
 
        # Draw and display the corners
        cv2.drawChessboardCorners(img, (7,6), corners2, ret)
        cv2.imwrite(f'out/{ospath.basename(fname)}', img)
        
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)


HEIGHT CALCULATION

In [41]:
def find_liquid_height(image, debug_name='out.jpg', glass_height_cm=10):
    """
    Detects the height of liquid in a glass image and calculates its real-world height in cm.

    Parameters:
    - image (numpy array): Input image of the glass.
    - debug_name (str): Filename for saving debug output images.
    - glass_height_cm (float): Real-world height of the glass in cm.

    Returns:
    - liquid_height_cm (float): Height of the liquid in cm.
    """

    # Resize image for easier processing (optional)
    scale_percent = 50  # Resize to 50% of original size
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    image = cv2.resize(image, (width, height))

    image = center_crop(image, width - 500, height - 100)

    # Convert to grayscale and apply Gaussian blur
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)

    # Detect edges using Sobel filter
    grad_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    gradient = cv2.magnitude(grad_x, grad_y)

    threshold = np.percentile(gradient, 99)
    _, glass_edges = cv2.threshold(np.uint8(gradient), threshold, 255, cv2.THRESH_BINARY)

    # Morphological operations to strengthen edges
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    glass_edges = cv2.morphologyEx(glass_edges, cv2.MORPH_CLOSE, kernel)
    glass_edges = cv2.morphologyEx(glass_edges, cv2.MORPH_OPEN, kernel)
    glass_edges = cv2.dilate(glass_edges, kernel, iterations=1)  # Increase iterations if needed

    cv2.imwrite(f'out/glass-{debug_name}', glass_edges)

    # Find glass contours
    contours, _ = cv2.findContours(glass_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    merged_contour = []
    for contour in contours:
        if cv2.contourArea(contour) > 500:
            merged_contour.extend(contour)  # Add the points from each contour

    # Convert the list of points into a numpy array
    merged_contour = np.array(merged_contour)

    hull = cv2.convexHull(merged_contour)

    cv2.imwrite(f'out/hull-{debug_name}', cv2.drawContours(np.zeros_like(image), [hull], -1, (255), thickness=2))

    # Ensure at least one contour is found
    if len(contours) == 0:
        raise ValueError("No glass detected.")

    # Identify the largest contour (assumed to be the glass)
    # glass_contour = max(contours, key=cv2.contourArea)

    # Bounding rectangle for the glass
    x, y, w, h = cv2.boundingRect(hull)
    glass_height_px = h  # Glass height in pixels

    # Crop the region of interest (glass)
    roi_gray = gray[y:y+h, x:x+w]

    # Segment the liquid using thresholding
    _, liquid_mask = cv2.threshold(roi_gray, 50, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    # Remove noise and small shadows
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    liquid_mask = cv2.erode(liquid_mask, kernel, iterations=5)
    liquid_mask = cv2.morphologyEx(liquid_mask, cv2.MORPH_CLOSE, kernel)
    liquid_mask = cv2.morphologyEx(liquid_mask, cv2.MORPH_OPEN, kernel)
    cv2.imwrite(f'out/liquid-mask{debug_name}', liquid_mask)

    # Step 2: Detect the top of the liquid in the mask
    liquid_y, liquid_x = np.where(liquid_mask > 0)
    if len(liquid_y) == 0 or len(liquid_x) == 0:
        raise ValueError("Liquid mask is empty.")

    # Liquid boundaries
    liquid_top = np.min(liquid_y)
    liquid_bottom = np.max(liquid_y)

    # Step 3: Exclude regions below the liquid surface
    # Define a threshold to ignore shadows (e.g., anything below 10% from the bottom)
    shadow_threshold = int((liquid_bottom - liquid_top) * 0.08)
    refined_mask = np.zeros_like(liquid_mask)
    refined_mask[liquid_top:liquid_bottom - shadow_threshold, :] = liquid_mask[liquid_top:liquid_bottom - shadow_threshold, :]

    # Step 4: Combine with the glass mask (optional refinement)
    liquid_only_mask = cv2.bitwise_and(refined_mask, liquid_mask)

    cv2.imwrite(f'out/liquid{debug_name}', liquid_mask)

    # Find contours for the liquid
    liquid_contours, _ = cv2.findContours(liquid_only_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Ensure at least one contour is found
    if len(liquid_contours) == 0:
        raise ValueError("No liquid detected.")
    
    # Identify the largest contour (assumed to be the liquid)
    liquid_contour = max(liquid_contours, key=cv2.contourArea)

    # Bounding rectangle for the liquid
    _, liquid_y, _, liquid_h = cv2.boundingRect(liquid_contour)
    liquid_height_px = liquid_h  # Liquid height in pixels

    # Calculate pixels per cm
    px_per_cm = glass_height_px / glass_height_cm

    # Calculate liquid height in cm
    liquid_height_cm = liquid_height_px / px_per_cm

    # Debug visualization
    debug_image = image.copy()
    cv2.rectangle(debug_image, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Glass
    cv2.rectangle(debug_image, (x, y + liquid_y), (x + w, y + liquid_y + liquid_h), (255, 0, 0), 2)  # Liquid
    cv2.imwrite(f'out/debug-{debug_name}', debug_image)

    return liquid_height_cm

# Percorso immagine
image_path = "images/A-A.jpg"

# Calcola l'altezza del liquido
try:
    image = cv2.imread(image_path)
    liquid_height = find_liquid_height(image)
    print(f"Altezza del liquido: {liquid_height:.2f} cm")
except Exception as e:
    print(f"Errore: {e}")


Altezza del liquido: 3.79 cm


FUSION

In [42]:
import re

filelist = glob.glob("images/?-?.jpg")

for file in filelist:
    # Percorso immagine
    tipe = re.search('([A-C]?)-[A-C]?', file).group(1)
    height = get_glass_height_cm(tipe)

    img = cv2.imread(file)
    h, w = img.shape[:2]
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
    # undistort
    dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
    # crop the image
    x, y, w, h = roi
    dst = dst[y:y+h, x:x+w]
    
    # Taglia il bordo del tavolo
    cv2.imwrite(f'out/calib-{ospath.basename(file)}', dst)

    # Calcola l'altezza del liquido
    try:
        liquid_height = find_liquid_height(img, debug_name=ospath.basename(file), glass_height_cm=height)
        print(f"Altezza del liquido in {ospath.basename(file)}: {round(liquid_height*10, 0)} mm")
    except Exception as e:
        print(f"Errore: {e}")

Altezza del liquido in A-A.jpg: 31.0 mm
Altezza del liquido in A-C.jpg: 48.0 mm
Altezza del liquido in A-F.jpg: 20.0 mm
Altezza del liquido in B-A.jpg: 34.0 mm
Altezza del liquido in B-C.jpg: 50.0 mm
Altezza del liquido in B-F.jpg: 22.0 mm
Altezza del liquido in C-A.jpg: 38.0 mm
Altezza del liquido in C-C.jpg: 50.0 mm
Altezza del liquido in C-F.jpg: 26.0 mm
